# 神经网络相似度测量综述 (Survey of Neural Network Similarity Measures)

**Based on:** Klabunde et al. (2025). "Similarity of Neural Network Models: A Survey of Functional and Representational Measures"

**作者整理 (Compiled by):** YueYueYYY

**日期 (Date):** 2025年12月

---

## 📋 目录 (Table of Contents)

1. [论文概述 (Paper Overview)](#section1)
2. [核心概念 (Core Concepts)](#section2)
3. [表示相似度测量 (Representational Similarity)](#section3)
4. [函数相似度测量 (Functional Similarity)](#section4)
5. [代码实现 (Code Implementation)](#section5)
6. [生物学应用示例 (Biological Applications)](#section6)
7. [总结与展望 (Summary)](#section7)

## Paper contents

## Explaination-what is neural network?/How It Works

---
<a id='section1'></a>
## 1. 论文概述 (Paper Overview)

### 1.1 研究动机 (Motivation)

**中文：**
- 测量神经网络相似度对理解和改进深度学习系统至关重要
- 应用领域：学习动态、模型鲁棒性、知识蒸馏、集成学习

**English:**
- Measuring similarity of neural networks is crucial for understanding and improving deep learning systems
- Applications: learning dynamics, robustness, knowledge distillation, ensemble learning

### 1.2 两种互补视角 (Two Complementary Perspectives)

```
┌─────────────────────────────────────────────────────────┐
│                   Neural Network f                       │
│                                                          │
│  Input X → [Layer 1] → [Layer 2] → ... → Output O      │
│               ↓           ↓                              │
│          Representation  Representation                  │
│               R₁          R₂                             │
└─────────────────────────────────────────────────────────┘

Representational Similarity: Compare R₁, R₂ (internal activations)
Functional Similarity:       Compare O (outputs)
```

### 1.3 论文贡献 (Key Contributions)

1. **系统性综述** - 53+ 相似度测量方法
2. **统一术语** - 标准化的数学定义
3. **实用指南** - 如何选择和应用这些方法
4. **开放问题** - 未解决的研究挑战

In [ ]:
# 安装必要的库 (Install necessary libraries)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体 (Set Chinese font)
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 设置随机种子 (Set random seed for reproducibility)
np.random.seed(42)

print("✅ Libraries loaded successfully!")
print("✅ 库加载成功！")

---
<a id='section2'></a>
## 2. 核心概念 (Core Concepts)

### 2.1 神经网络的数学定义 (Mathematical Definition)

**论文 Equation (1):**

$$f = f^{(L)} \circ f^{(L-1)} \circ \cdots \circ f^{(1)}$$

其中 (where):
- $f^{(l)}: \mathbb{R}^{D^{(l-1)}} \rightarrow \mathbb{R}^{D^{(l)}}$ 是第 $l$ 层
- $D^{(l)}$ 是第 $l$ 层的神经元数量
- $L$ 是总层数

### 2.2 表示 (Representation)

**论文 Equation (2):**

$$R := R^{(l)} = \left(f^{(l)} \circ f^{(l-1)} \circ \cdots \circ f^{(1)}\right)(X) \in \mathbb{R}^{N \times D}$$

其中 (where):
- $N$ = 输入样本数量 (number of input instances)
- $D$ = 神经元数量 (number of neurons)
- $R_i$ = 第 $i$ 个样本的表示 (representation of instance $i$)

In [1]:
# 示例：创建简单的神经网络表示
# Example: Create simple neural network representations

def simple_neural_network(X, W1, b1, W2, b2):
    """
    简单的两层神经网络
    Simple two-layer neural network
    
    f(x) = W2 * ReLU(W1 * x + b1) + b2
    """
    # Layer 1: 第一层
    h1 = np.maximum(0, X @ W1 + b1)  # ReLU activation
    
    # Layer 2: 第二层
    output = h1 @ W2 + b2
    
    return h1, output  # 返回中间表示和输出

# 创建模拟数据 (Create synthetic data)
N = 100  # 样本数 (number of samples)
input_dim = 10
hidden_dim = 5
output_dim = 3

# 输入数据 (Input data)
X = np.random.randn(N, input_dim)

# 模型A的参数 (Model A parameters)
W1_A = np.random.randn(input_dim, hidden_dim) * 0.1
b1_A = np.zeros(hidden_dim)
W2_A = np.random.randn(hidden_dim, output_dim) * 0.1
b2_A = np.zeros(output_dim)

# 模型B的参数 (Model B parameters)
W1_B = np.random.randn(input_dim, hidden_dim) * 0.1
b1_B = np.zeros(hidden_dim)
W2_B = np.random.randn(hidden_dim, output_dim) * 0.1
b2_B = np.zeros(output_dim)

# 获取表示 (Get representations)
R_A, O_A = simple_neural_network(X, W1_A, b1_A, W2_A, b2_A)
R_B, O_B = simple_neural_network(X, W1_B, b1_B, W2_B, b2_B)

print(f"表示维度 (Representation shape): {R_A.shape}")
print(f"输出维度 (Output shape): {O_A.shape}")
print(f"\nModel A 表示统计 (Representation statistics):")
print(f"  Mean: {R_A.mean():.4f}")
print(f"  Std:  {R_A.std():.4f}")

NameError: name 'np' is not defined

In [ ]:
# 可视化表示 (Visualize representations)
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot 1: Model A representation
im1 = axes[0].imshow(R_A.T, aspect='auto', cmap='viridis')
axes[0].set_title('Model A Representation\n模型A的表示', fontsize=12)
axes[0].set_xlabel('Samples (样本)')
axes[0].set_ylabel('Neurons (神经元)')
plt.colorbar(im1, ax=axes[0])

# Plot 2: Model B representation
im2 = axes[1].imshow(R_B.T, aspect='auto', cmap='viridis')
axes[1].set_title('Model B Representation\n模型B的表示', fontsize=12)
axes[1].set_xlabel('Samples (样本)')
axes[1].set_ylabel('Neurons (神经元)')
plt.colorbar(im2, ax=axes[1])

# Plot 3: Difference
diff = np.abs(R_A - R_B)
im3 = axes[2].imshow(diff.T, aspect='auto', cmap='Reds')
axes[2].set_title('Absolute Difference\n绝对差异', fontsize=12)
axes[2].set_xlabel('Samples (样本)')
axes[2].set_ylabel('Neurons (神经元)')
plt.colorbar(im3, ax=axes[2])

plt.tight_layout()
plt.show()

print(f"平均差异 (Mean difference): {diff.mean():.4f}")

---
<a id='section3'></a>
## 3. 表示相似度测量 (Representational Similarity Measures)

### 3.1 测量分类 (Categories of Measures)

论文将表示相似度测量分为6大类 (The paper categorizes measures into 6 types):

1. **CCA-based (基于典型相关分析)** - CCA, SVCCA, PWCCA
2. **Alignment-based (基于对齐)** - Procrustes, Linear Regression
3. **RSM-based (基于表示相似性矩阵)** - CKA, RSA, dCor
4. **Neighbor-based (基于近邻)** - k-NN Jaccard
5. **Topology-based (基于拓扑)** - Geometry Score, IMD
6. **Statistics-based (基于统计)** - Intrinsic Dimension, Magnitude

### 3.2 关键概念：不变性 (Key Concept: Invariances)

测量方法对不同变换的不变性 (Invariance to transformations):

| 变换 Transformation | 符号 Symbol | 说明 Description |
|-------------------|------------|------------------|
| 排列 Permutation | PT | 重排神经元顺序 Reorder neurons |
| 正交 Orthogonal | OT | 旋转和反射 Rotation & reflection |
| 缩放 Isotropic Scaling | IS | 统一缩放 Uniform scaling |
| 线性 Invertible Linear | ILT | 可逆线性变换 Invertible linear |
| 平移 Translation | TR | 平移 Translation |
| 仿射 Affine | AT | 仿射变换 Affine transformation |

### 3.3 重点方法1：CKA (Centered Kernel Alignment)

**最流行的表示相似度测量方法！(Most popular representational similarity measure!)**

**论文 Equation (28):**

$$m_{\text{CKA}}(R, R') = \frac{\text{HSIC}(S, S')}{\sqrt{\text{HSIC}(S, S) \cdot \text{HSIC}(S', S')}}$$

其中 (where):
- $S = RR^T$ (线性核 Linear kernel)
- $\text{HSIC}(S, S') = \frac{1}{(N-1)^2} \text{tr}(SH_NS'H_N)$
- $H_N = I_N - \frac{1}{N}\mathbf{1}_N\mathbf{1}_N^T$ (中心化矩阵 Centering matrix)

**性质 (Properties):**
- 范围 Range: [0, 1]
- 1 = 完全相似 (Perfectly similar)
- 0 = 完全不同 (Completely different)
- 不变性 Invariances: 正交变换 + 等向缩放 (OT + IS)

In [ ]:
def center_kernel(K):
    """
    中心化核矩阵
    Center the kernel matrix
    """
    N = K.shape[0]
    H = np.eye(N) - np.ones((N, N)) / N
    return H @ K @ H

def hsic(K, L):
    """
    Hilbert-Schmidt Independence Criterion
    """
    N = K.shape[0]
    K_c = center_kernel(K)
    L_c = center_kernel(L)
    return np.trace(K_c @ L_c) / ((N - 1) ** 2)

def cka(R1, R2, kernel='linear'):
    """
    计算CKA相似度
    Compute CKA similarity
    
    参数 Parameters:
        R1, R2: 表示矩阵 (N x D) Representation matrices
        kernel: 'linear' or 'rbf'
    
    返回 Returns:
        CKA similarity score [0, 1]
    """
    if kernel == 'linear':
        K = R1 @ R1.T  # Linear kernel
        L = R2 @ R2.T
    elif kernel == 'rbf':
        # RBF kernel with median heuristic
        sigma1 = np.median(cdist(R1, R1))
        sigma2 = np.median(cdist(R2, R2))
        K = np.exp(-cdist(R1, R1)**2 / (2 * sigma1**2))
        L = np.exp(-cdist(R2, R2)**2 / (2 * sigma2**2))
    
    hsic_kl = hsic(K, L)
    hsic_kk = hsic(K, K)
    hsic_ll = hsic(L, L)
    
    cka_score = hsic_kl / np.sqrt(hsic_kk * hsic_ll)
    return cka_score

# 测试CKA (Test CKA)
print("=" * 50)
print("CKA Similarity Test / CKA相似度测试")
print("=" * 50)

# Test 1: 相同的表示 (Identical representations)
cka_identical = cka(R_A, R_A)
print(f"\n1. 相同表示 (Identical): CKA = {cka_identical:.4f}")

# Test 2: 不同的表示 (Different representations)
cka_different = cka(R_A, R_B)
print(f"2. 不同表示 (Different): CKA = {cka_different:.4f}")

# Test 3: 添加噪声 (Add noise)
R_A_noisy = R_A + np.random.randn(*R_A.shape) * 0.1
cka_noisy = cka(R_A, R_A_noisy)
print(f"3. 添加噪声 (With noise): CKA = {cka_noisy:.4f}")

# Test 4: 旋转不变性 (Rotation invariance)
Q = np.linalg.qr(np.random.randn(R_A.shape[1], R_A.shape[1]))[0]  # Random orthogonal matrix
R_A_rotated = R_A @ Q
cka_rotated = cka(R_A, R_A_rotated)
print(f"4. 旋转后 (After rotation): CKA = {cka_rotated:.4f}")

print("\n✅ CKA对正交变换保持不变！")
print("✅ CKA is invariant to orthogonal transformations!")

### 3.4 重点方法2：典型相关分析 (Canonical Correlation Analysis)

**论文 Equation (10-12)**

CCA找到最大化相关性的权重向量 (CCA finds weight vectors that maximize correlation):

$$\rho_i = \max_{w_R, w_{R'}} \frac{\langle Rw_R, R'w_{R'} \rangle}{\|Rw_R\| \cdot \|R'w_{R'}\|}$$

**Mean CCA:**
$$m_{\text{CCA}}(R, R') = \frac{1}{D} \sum_{i=1}^{D} \rho_i$$

In [ ]:
def mean_cca(R1, R2, n_components=None):
    """
    计算平均CCA相似度
    Compute mean CCA similarity
    """
    if n_components is None:
        n_components = min(R1.shape[1], R2.shape[1])
    
    # 中心化 (Center the data)
    R1_centered = R1 - R1.mean(axis=0)
    R2_centered = R2 - R2.mean(axis=0)
    
    # 使用sklearn的CCA (Use sklearn's CCA)
    cca = CCA(n_components=n_components)
    cca.fit(R1_centered, R2_centered)
    
    # 转换 (Transform)
    R1_c, R2_c = cca.transform(R1_centered, R2_centered)
    
    # 计算相关系数 (Compute correlations)
    correlations = []
    for i in range(n_components):
        corr = np.corrcoef(R1_c[:, i], R2_c[:, i])[0, 1]
        correlations.append(corr)
    
    return np.mean(correlations), correlations

# 计算CCA (Compute CCA)
mean_cca_score, cca_correlations = mean_cca(R_A, R_B)

print(f"Mean CCA: {mean_cca_score:.4f}")
print(f"\n各维度的典型相关系数 (Canonical correlations):")
for i, corr in enumerate(cca_correlations, 1):
    print(f"  维度 {i}: {corr:.4f}")

### 3.5 重点方法3：Procrustes距离 (Procrustes Distance)

**论文 Equation (15)**

寻找最优正交变换来对齐表示 (Find optimal orthogonal transformation to align representations):

$$m_{\text{Procrustes}}(R, R') = \min_{Q \in O(D)} \|RQ - R'\|_F$$

其中 $O(D)$ 是正交矩阵群 (orthogonal group)

In [ ]:
def orthogonal_procrustes(R1, R2):
    """
    计算正交Procrustes距离
    Compute orthogonal Procrustes distance
    
    使用SVD求解: R1 @ Q ≈ R2
    Solve using SVD: R1 @ Q ≈ R2
    """
    # SVD分解 (SVD decomposition)
    U, _, Vt = np.linalg.svd(R1.T @ R2)
    
    # 最优正交矩阵 (Optimal orthogonal matrix)
    Q_opt = U @ Vt
    
    # 计算距离 (Compute distance)
    R1_aligned = R1 @ Q_opt
    distance = np.linalg.norm(R1_aligned - R2, 'fro')
    
    return distance, Q_opt, R1_aligned

# 测试 (Test)
proc_dist, Q_opt, R_A_aligned = orthogonal_procrustes(R_A, R_B)

print(f"Procrustes距离 (Procrustes distance): {proc_dist:.4f}")
print(f"\n对齐前的差异 (Before alignment): {np.linalg.norm(R_A - R_B, 'fro'):.4f}")
print(f"对齐后的差异 (After alignment):  {np.linalg.norm(R_A_aligned - R_B, 'fro'):.4f}")

### 3.6 比较不同的相似度测量方法

让我们在不同场景下比较这些方法 (Let's compare these methods in different scenarios)

In [ ]:
# 创建不同场景的表示对 (Create representation pairs for different scenarios)
scenarios = {
    '1. Identical\n相同': (R_A, R_A),
    '2. Random\n随机': (R_A, R_B),
    '3. Rotated\n旋转': (R_A, R_A @ Q),
    '4. Noisy\n噪声': (R_A, R_A + np.random.randn(*R_A.shape) * 0.1),
    '5. Scaled\n缩放': (R_A, R_A * 2.0),
}

# 计算所有方法的相似度 (Compute similarity for all methods)
results = []
for name, (R1, R2) in scenarios.items():
    cka_score = cka(R1, R2)
    mean_cca_score, _ = mean_cca(R1, R2)
    proc_dist, _, _ = orthogonal_procrustes(R1, R2)
    # 归一化Procrustes距离 (Normalize Procrustes distance)
    proc_similarity = 1 / (1 + proc_dist)
    
    results.append({
        'Scenario': name,
        'CKA': cka_score,
        'Mean CCA': mean_cca_score,
        'Procrustes': proc_similarity
    })

# 可视化结果 (Visualize results)
import pandas as pd

df = pd.DataFrame(results)
print("\n" + "="*60)
print("相似度测量比较 (Similarity Measure Comparison)")
print("="*60)
print(df.to_string(index=False))

# 绘图 (Plot)
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(scenarios))
width = 0.25

ax.bar(x - width, df['CKA'], width, label='CKA', alpha=0.8)
ax.bar(x, df['Mean CCA'], width, label='Mean CCA', alpha=0.8)
ax.bar(x + width, df['Procrustes'], width, label='Procrustes (norm)', alpha=0.8)

ax.set_xlabel('Scenarios (场景)', fontsize=12)
ax.set_ylabel('Similarity (相似度)', fontsize=12)
ax.set_title('Comparison of Representational Similarity Measures\n表示相似度测量方法比较', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(df['Scenario'], rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)
ax.set_ylim([0, 1.1])

plt.tight_layout()
plt.show()

print("\n🔍 观察 (Observations):")
print("  • CKA对旋转和缩放不变 (CKA is invariant to rotation and scaling)")
print("  • Procrustes对旋转不变 (Procrustes is invariant to rotation)")
print("  • 所有方法对噪声都敏感 (All methods are sensitive to noise)")

---
<a id='section4'></a>
## 4. 函数相似度测量 (Functional Similarity Measures)

函数相似度关注**模型的输出行为** (Functional similarity focuses on **model output behavior**)

### 4.1 测量分类 (Categories)

1. **Performance-based (基于性能)** - 比较准确率
2. **Hard prediction-based (基于硬预测)** - 比较分类结果
3. **Soft prediction-based (基于软预测)** - 比较概率分布
4. **Gradient-based (基于梯度)** - 比较梯度和对抗样本

### 4.2 不一致度 (Disagreement)

**论文 Equation (51)** - 最简单且最常用的方法

$$m_{\text{Dis}}(O, O') = \frac{1}{N} \sum_{i=1}^{N} \mathbb{1}\{\arg\max_j O_{i,j} \neq \arg\max_j O'_{i,j}\}$$

- 范围 Range: [0, 1]
- 0 = 完全一致 (Perfect agreement)
- 1 = 完全不同 (Complete disagreement)

In [ ]:
def disagreement(O1, O2):
    """
    计算预测不一致度
    Compute prediction disagreement
    
    参数 Parameters:
        O1, O2: 输出矩阵 (N x C) Output matrices
    
    返回 Returns:
        Disagreement rate [0, 1]
    """
    pred1 = np.argmax(O1, axis=1)
    pred2 = np.argmax(O2, axis=1)
    return np.mean(pred1 != pred2)

# 创建模拟的分类输出 (Create synthetic classification outputs)
# 假设3类分类问题 (Assume 3-class classification)
N = 100
C = 3

# 模型A的输出 (Model A outputs)
logits_A = np.random.randn(N, C)
probs_A = np.exp(logits_A) / np.exp(logits_A).sum(axis=1, keepdims=True)

# 模型B的输出 (Model B outputs - somewhat similar to A)
logits_B = logits_A + np.random.randn(N, C) * 0.5
probs_B = np.exp(logits_B) / np.exp(logits_B).sum(axis=1, keepdims=True)

# 计算不一致度 (Compute disagreement)
dis = disagreement(probs_A, probs_B)
print(f"Disagreement (不一致度): {dis:.4f}")
print(f"Agreement (一致度): {1-dis:.4f}")

### 4.3 Jensen-Shannon散度 (Jensen-Shannon Divergence)

**论文 Equation (60)** - 比较概率分布

$$m_{\text{JSD}}(O, O') = \frac{1}{2N} \sum_{i=1}^{N} [\text{KL}(O_i || \bar{O}_i) + \text{KL}(O'_i || \bar{O}_i)]$$

其中 $\bar{O} = \frac{O + O'}{2}$ (平均输出)

In [ ]:
def kl_divergence(p, q, epsilon=1e-10):
    """KL散度 (KL divergence)"""
    p = np.clip(p, epsilon, 1)
    q = np.clip(q, epsilon, 1)
    return np.sum(p * np.log(p / q))

def jensen_shannon_divergence(O1, O2):
    """
    计算Jensen-Shannon散度
    Compute Jensen-Shannon divergence
    """
    O_avg = (O1 + O2) / 2
    jsd = 0
    for i in range(len(O1)):
        jsd += kl_divergence(O1[i], O_avg[i]) + kl_divergence(O2[i], O_avg[i])
    return jsd / (2 * len(O1))

# 计算JSD (Compute JSD)
jsd = jensen_shannon_divergence(probs_A, probs_B)
print(f"Jensen-Shannon Divergence: {jsd:.4f}")
print(f"\n💡 JSD = 0 表示完全相同 (JSD = 0 means identical)")
print(f"   JSD越大,分布差异越大 (Larger JSD means more different)")

### 4.4 函数相似度综合比较

让我们比较不同的函数相似度测量方法 (Let's compare different functional similarity measures)

In [ ]:
def cosine_similarity_outputs(O1, O2):
    """计算输出的余弦相似度 (Compute cosine similarity of outputs)"""
    cos_sims = []
    for i in range(len(O1)):
        cos_sim = np.dot(O1[i], O2[i]) / (np.linalg.norm(O1[i]) * np.linalg.norm(O2[i]))
        cos_sims.append(cos_sim)
    return np.mean(cos_sims)

# 创建不同相似度的模型对 (Create model pairs with different similarities)
scenarios_func = {
    'Identical\n相同': probs_A,
    'Similar\n相似': probs_A + np.random.randn(N, C) * 0.1,
    'Moderate\n中等': probs_A + np.random.randn(N, C) * 0.5,
    'Different\n不同': np.random.rand(N, C),
}

# 归一化概率 (Normalize probabilities)
for key in scenarios_func:
    if key != 'Identical\n相同':
        scenarios_func[key] = np.abs(scenarios_func[key])
        scenarios_func[key] = scenarios_func[key] / scenarios_func[key].sum(axis=1, keepdims=True)

# 计算所有指标 (Compute all metrics)
results_func = []
for name, O2 in scenarios_func.items():
    dis = disagreement(probs_A, O2)
    jsd = jensen_shannon_divergence(probs_A, O2)
    cos_sim = cosine_similarity_outputs(probs_A, O2)
    
    results_func.append({
        'Scenario': name,
        'Disagreement': dis,
        'JSD': jsd,
        'Cos_Sim': cos_sim
    })

# 显示结果 (Display results)
df_func = pd.DataFrame(results_func)
print("\n" + "="*60)
print("函数相似度测量比较 (Functional Similarity Comparison)")
print("="*60)
print(df_func.to_string(index=False))

# 可视化 (Visualize)
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Disagreement
axes[0].bar(range(len(df_func)), df_func['Disagreement'], color='coral', alpha=0.7)
axes[0].set_title('Disagreement\n不一致度', fontsize=12)
axes[0].set_xticks(range(len(df_func)))
axes[0].set_xticklabels(df_func['Scenario'], rotation=45, ha='right')
axes[0].set_ylabel('Score')
axes[0].grid(axis='y', alpha=0.3)
axes[0].set_ylim([0, 1])

# JSD
axes[1].bar(range(len(df_func)), df_func['JSD'], color='skyblue', alpha=0.7)
axes[1].set_title('Jensen-Shannon Divergence\nJSD散度', fontsize=12)
axes[1].set_xticks(range(len(df_func)))
axes[1].set_xticklabels(df_func['Scenario'], rotation=45, ha='right')
axes[1].set_ylabel('Score')
axes[1].grid(axis='y', alpha=0.3)

# Cosine Similarity
axes[2].bar(range(len(df_func)), df_func['Cos_Sim'], color='lightgreen', alpha=0.7)
axes[2].set_title('Cosine Similarity\n余弦相似度', fontsize=12)
axes[2].set_xticks(range(len(df_func)))
axes[2].set_xticklabels(df_func['Scenario'], rotation=45, ha='right')
axes[2].set_ylabel('Score')
axes[2].grid(axis='y', alpha=0.3)
axes[2].set_ylim([0, 1])

plt.tight_layout()
plt.show()

---
<a id='section5'></a>
## 5. 完整的相似度分析流程

### Complete Similarity Analysis Pipeline

In [ ]:
class NeuralNetworkSimilarity:
    """
    神经网络相似度分析工具类
    Neural Network Similarity Analysis Tool
    """
    
    def __init__(self, model_A, model_B, X):
        """
        参数 Parameters:
            model_A, model_B: 两个神经网络模型
            X: 输入数据
        """
        self.model_A = model_A
        self.model_B = model_B
        self.X = X
        self.results = {}
    
    def compute_representational_similarity(self, layer='hidden'):
        """
        计算表示相似度
        Compute representational similarity
        """
        print("\n" + "="*60)
        print("表示相似度分析 (Representational Similarity Analysis)")
        print("="*60)
        
        # 提取表示 (Extract representations)
        if layer == 'hidden':
            R_A, _ = self.model_A(self.X)
            R_B, _ = self.model_B(self.X)
        
        # 计算各种相似度 (Compute various similarities)
        self.results['CKA'] = cka(R_A, R_B)
        self.results['Mean_CCA'], _ = mean_cca(R_A, R_B)
        self.results['Procrustes'], _, _ = orthogonal_procrustes(R_A, R_B)
        
        print(f"\n✅ CKA Similarity:         {self.results['CKA']:.4f}")
        print(f"✅ Mean CCA:              {self.results['Mean_CCA']:.4f}")
        print(f"✅ Procrustes Distance:  {self.results['Procrustes']:.4f}")
        
        return self.results
    
    def compute_functional_similarity(self):
        """
        计算函数相似度
        Compute functional similarity
        """
        print("\n" + "="*60)
        print("函数相似度分析 (Functional Similarity Analysis)")
        print("="*60)
        
        # 获取输出 (Get outputs)
        _, O_A = self.model_A(self.X)
        _, O_B = self.model_B(self.X)
        
        # 转换为概率 (Convert to probabilities)
        probs_A = np.exp(O_A) / np.exp(O_A).sum(axis=1, keepdims=True)
        probs_B = np.exp(O_B) / np.exp(O_B).sum(axis=1, keepdims=True)
        
        # 计算各种相似度 (Compute various similarities)
        self.results['Disagreement'] = disagreement(probs_A, probs_B)
        self.results['JSD'] = jensen_shannon_divergence(probs_A, probs_B)
        self.results['Cosine_Sim'] = cosine_similarity_outputs(probs_A, probs_B)
        
        print(f"\n✅ Disagreement:         {self.results['Disagreement']:.4f}")
        print(f"✅ JSD:                  {self.results['JSD']:.4f}")
        print(f"✅ Output Cosine Sim:    {self.results['Cosine_Sim']:.4f}")
        
        return self.results
    
    def visualize_summary(self):
        """
        可视化总结
        Visualize summary
        """
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Representational similarity
        rep_measures = ['CKA', 'Mean_CCA']
        rep_scores = [self.results.get(m, 0) for m in rep_measures]
        axes[0].bar(rep_measures, rep_scores, color=['#FF6B6B', '#4ECDC4'], alpha=0.8)
        axes[0].set_title('Representational Similarity\n表示相似度', fontsize=14, fontweight='bold')
        axes[0].set_ylabel('Similarity Score', fontsize=12)
        axes[0].set_ylim([0, 1.1])
        axes[0].grid(axis='y', alpha=0.3)
        for i, v in enumerate(rep_scores):
            axes[0].text(i, v + 0.05, f'{v:.3f}', ha='center', fontweight='bold')
        
        # Functional similarity
        func_measures = ['Agreement\n(1-Dis)', 'Cosine_Sim']
        func_scores = [1 - self.results.get('Disagreement', 0), 
                      self.results.get('Cosine_Sim', 0)]
        axes[1].bar(func_measures, func_scores, color=['#95E1D3', '#F38181'], alpha=0.8)
        axes[1].set_title('Functional Similarity\n函数相似度', fontsize=14, fontweight='bold')
        axes[1].set_ylabel('Similarity Score', fontsize=12)
        axes[1].set_ylim([0, 1.1])
        axes[1].grid(axis='y', alpha=0.3)
        for i, v in enumerate(func_scores):
            axes[1].text(i, v + 0.05, f'{v:.3f}', ha='center', fontweight='bold')
        
        plt.tight_layout()
        plt.show()

# 测试完整流程 (Test complete pipeline)
print("\n" + "#"*60)
print("# 完整的神经网络相似度分析 (Complete NN Similarity Analysis) #")
print("#"*60)

# 创建模型函数 (Create model functions)
def model_A_func(X):
    return simple_neural_network(X, W1_A, b1_A, W2_A, b2_A)

def model_B_func(X):
    return simple_neural_network(X, W1_B, b1_B, W2_B, b2_B)

# 运行分析 (Run analysis)
analyzer = NeuralNetworkSimilarity(model_A_func, model_B_func, X)
analyzer.compute_representational_similarity()
analyzer.compute_functional_similarity()
analyzer.visualize_summary()

---
<a id='section6'></a>
## 6. 生物学应用示例 (Biological Applications)

### 场景：比较重金属毒性预测模型
### Scenario: Comparing Heavy Metal Toxicity Prediction Models

**研究问题 (Research Question):**
- 镉(Cd)和汞(Hg)是否通过相似的分子机制产生毒性?
- Do cadmium (Cd) and mercury (Hg) produce toxicity through similar molecular mechanisms?

In [ ]:
# 模拟基因表达数据 (Simulate gene expression data)
print("🧬 生物学应用示例：重金属毒性研究")
print("🧬 Biological Application: Heavy Metal Toxicity Study")
print("="*60)

# 数据设置 (Data setup)
n_genes = 1000  # 基因数量
n_samples = 200  # 样本数量
hidden_dim_bio = 50  # 隐藏层维度

# 生成基因表达数据 (Generate gene expression data)
# 正常样本 + 镉暴露 + 汞暴露
X_bio = np.random.lognormal(mean=5, sigma=2, size=(n_samples, n_genes))

# 真实标签 (True labels): 0=正常, 1=低毒, 2=高毒
y_true = np.random.choice([0, 1, 2], size=n_samples, p=[0.5, 0.3, 0.2])

print(f"\n数据维度 (Data dimensions):")
print(f"  基因数 (Genes): {n_genes}")
print(f"  样本数 (Samples): {n_samples}")
print(f"  类别分布 (Class distribution): {np.bincount(y_true)}")

# 创建两个模型：分别在Cd和Hg数据上训练
# Create two models: trained on Cd and Hg data respectively

# 模型参数 (Model parameters)
np.random.seed(42)
W1_Cd = np.random.randn(n_genes, hidden_dim_bio) * 0.01
b1_Cd = np.zeros(hidden_dim_bio)
W2_Cd = np.random.randn(hidden_dim_bio, 3) * 0.01
b2_Cd = np.zeros(3)

# Hg模型参数 (Hg model - somewhat similar to Cd)
np.random.seed(43)
W1_Hg = W1_Cd + np.random.randn(n_genes, hidden_dim_bio) * 0.005  # 相似但不完全相同
b1_Hg = b1_Cd
W2_Hg = W2_Cd + np.random.randn(hidden_dim_bio, 3) * 0.005
b2_Hg = b2_Cd

# 获取表示和输出 (Get representations and outputs)
R_Cd, O_Cd = simple_neural_network(X_bio, W1_Cd, b1_Cd, W2_Cd, b2_Cd)
R_Hg, O_Hg = simple_neural_network(X_bio, W1_Hg, b1_Hg, W2_Hg, b2_Hg)

print(f"\n表示维度 (Representation shape): {R_Cd.shape}")

In [ ]:
# 计算相似度 (Compute similarities)
print("\n" + "="*60)
print("镉(Cd) vs 汞(Hg)模型相似度分析")
print("Cd vs Hg Model Similarity Analysis")
print("="*60)

# 表示相似度 (Representational similarity)
cka_bio = cka(R_Cd, R_Hg)
mean_cca_bio, cca_corrs_bio = mean_cca(R_Cd, R_Hg, n_components=5)
proc_bio, _, _ = orthogonal_procrustes(R_Cd, R_Hg)

print("\n🔬 表示相似度 (Representational Similarity):")
print(f"  CKA:       {cka_bio:.4f}")
print(f"  Mean CCA:  {mean_cca_bio:.4f}")
print(f"  Procrustes: {proc_bio:.4f}")

# 函数相似度 (Functional similarity)
probs_Cd = np.exp(O_Cd) / np.exp(O_Cd).sum(axis=1, keepdims=True)
probs_Hg = np.exp(O_Hg) / np.exp(O_Hg).sum(axis=1, keepdims=True)

dis_bio = disagreement(probs_Cd, probs_Hg)
jsd_bio = jensen_shannon_divergence(probs_Cd, probs_Hg)

print("\n🎯 函数相似度 (Functional Similarity):")
print(f"  Disagreement: {dis_bio:.4f}")
print(f"  Agreement:    {1-dis_bio:.4f}")
print(f"  JSD:          {jsd_bio:.4f}")

# 生物学解释 (Biological interpretation)
print("\n" + "="*60)
print("🔍 生物学解释 (Biological Interpretation)")
print("="*60)

if cka_bio > 0.7:
    print("\n✅ 高表示相似度 (CKA > 0.7):")
    print("   两种重金属可能激活相似的分子通路")
    print("   Both metals likely activate similar molecular pathways")
elif cka_bio > 0.4:
    print("\n⚠️  中等表示相似度 (0.4 < CKA < 0.7):")
    print("   部分共享机制,但也存在独特的作用途径")
    print("   Partially shared mechanisms with unique pathways")
else:
    print("\n❌ 低表示相似度 (CKA < 0.4):")
    print("   两种重金属通过不同的分子机制产生毒性")
    print("   Different molecular mechanisms of toxicity")

if dis_bio < 0.2:
    print("\n✅ 高预测一致性 (Agreement > 0.8):")
    print("   模型对毒性水平的判断高度一致")
    print("   High agreement in toxicity level predictions")
else:
    print("\n⚠️  预测存在差异:")
    print("   尽管内部表示相似,但最终毒性表现可能不同")
    print("   Despite similar representations, final toxicity may differ")

In [ ]:
# 可视化生物学分析结果 (Visualize biological analysis)
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. 表示的PCA可视化 (PCA visualization of representations)
ax1 = fig.add_subplot(gs[0, :])
pca = PCA(n_components=2)
R_Cd_pca = pca.fit_transform(R_Cd)
R_Hg_pca = pca.transform(R_Hg)

scatter1 = ax1.scatter(R_Cd_pca[:, 0], R_Cd_pca[:, 1], 
                       c=y_true, cmap='viridis', alpha=0.6, s=50, 
                       marker='o', label='Cd Model', edgecolors='black', linewidth=0.5)
scatter2 = ax1.scatter(R_Hg_pca[:, 0], R_Hg_pca[:, 1], 
                       c=y_true, cmap='viridis', alpha=0.6, s=50, 
                       marker='^', label='Hg Model', edgecolors='red', linewidth=0.5)
ax1.set_xlabel('PC1', fontsize=12)
ax1.set_ylabel('PC2', fontsize=12)
ax1.set_title('Representation Space (PCA)\n表示空间(PCA可视化)', fontsize=13, fontweight='bold')
ax1.legend(loc='upper right')
ax1.grid(alpha=0.3)
cbar = plt.colorbar(scatter1, ax=ax1)
cbar.set_label('Toxicity Level (毒性水平)', rotation=270, labelpad=20)

# 2. 相似度得分 (Similarity scores)
ax2 = fig.add_subplot(gs[1, 0])
rep_metrics = ['CKA', 'Mean\nCCA']
rep_values = [cka_bio, mean_cca_bio]
bars = ax2.bar(rep_metrics, rep_values, color=['#FF6B6B', '#4ECDC4'], alpha=0.8, edgecolor='black')
ax2.set_ylim([0, 1.1])
ax2.set_title('Representational\nSimilarity\n表示相似度', fontsize=11, fontweight='bold')
ax2.set_ylabel('Score')
ax2.grid(axis='y', alpha=0.3)
for bar, val in zip(bars, rep_values):
    ax2.text(bar.get_x() + bar.get_width()/2, val + 0.05, 
             f'{val:.3f}', ha='center', fontweight='bold', fontsize=10)

# 3. 函数相似度 (Functional similarity)
ax3 = fig.add_subplot(gs[1, 1])
func_metrics = ['Agreement\n一致度', 'Output\nCos Sim']
func_values = [1-dis_bio, cosine_similarity_outputs(probs_Cd, probs_Hg)]
bars = ax3.bar(func_metrics, func_values, color=['#95E1D3', '#F38181'], alpha=0.8, edgecolor='black')
ax3.set_ylim([0, 1.1])
ax3.set_title('Functional\nSimilarity\n函数相似度', fontsize=11, fontweight='bold')
ax3.set_ylabel('Score')
ax3.grid(axis='y', alpha=0.3)
for bar, val in zip(bars, func_values):
    ax3.text(bar.get_x() + bar.get_width()/2, val + 0.05, 
             f'{val:.3f}', ha='center', fontweight='bold', fontsize=10)

# 4. 典型相关系数 (Canonical correlations)
ax4 = fig.add_subplot(gs[1, 2])
ax4.plot(range(1, len(cca_corrs_bio)+1), cca_corrs_bio, 
         'o-', color='purple', linewidth=2, markersize=8)
ax4.set_xlabel('Component', fontsize=11)
ax4.set_ylabel('Correlation', fontsize=11)
ax4.set_title('Canonical\nCorrelations\n典型相关系数', fontsize=11, fontweight='bold')
ax4.grid(alpha=0.3)
ax4.set_ylim([0, 1.1])

# 5. 预测分布比较 (Prediction distribution comparison)
ax5 = fig.add_subplot(gs[2, :])
pred_Cd = np.argmax(probs_Cd, axis=1)
pred_Hg = np.argmax(probs_Hg, axis=1)

# 混淆矩阵可视化 (Confusion matrix visualization)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(pred_Cd, pred_Hg)
im = ax5.imshow(cm, cmap='YlOrRd', aspect='auto')
ax5.set_xlabel('Hg Model Prediction (汞模型预测)', fontsize=12)
ax5.set_ylabel('Cd Model Prediction (镉模型预测)', fontsize=12)
ax5.set_title('Prediction Agreement Matrix\n预测一致性矩阵', fontsize=13, fontweight='bold')
ax5.set_xticks([0, 1, 2])
ax5.set_yticks([0, 1, 2])
ax5.set_xticklabels(['Normal\n正常', 'Low Tox\n低毒', 'High Tox\n高毒'])
ax5.set_yticklabels(['Normal\n正常', 'Low Tox\n低毒', 'High Tox\n高毒'])

# 添加数值标签 (Add numerical labels)
for i in range(3):
    for j in range(3):
        text = ax5.text(j, i, cm[i, j], ha="center", va="center", 
                       color="white" if cm[i, j] > cm.max()/2 else "black",
                       fontsize=14, fontweight='bold')

plt.colorbar(im, ax=ax5, label='Sample Count (样本数)')

plt.suptitle('Cd vs Hg Toxicity Model Comparison\n镉 vs 汞毒性模型综合比较', 
             fontsize=16, fontweight='bold', y=0.995)
plt.show()

print("\n✅ 可视化完成！ (Visualization complete!)")

---
<a id='section7'></a>
## 7. 总结与关键要点 (Summary & Key Takeaways)

### 7.1 测量方法选择指南 (Guide for Selecting Measures)

#### 表示相似度 (Representational Similarity):

| 方法 Method | 优点 Advantages | 缺点 Disadvantages | 推荐场景 Recommended Use |
|------------|----------------|-------------------|-------------------------|
| **CKA** | • 最流行<br>• 对旋转/缩放不变<br>• 范围[0,1]易解释 | • 对单个样本改变敏感<br>• 计算复杂度O(N²) | • 一般用途<br>• 比较不同架构<br>• 层级比较 |
| **CCA** | • 考虑线性关系<br>• 维度自适应 | • 需要D≥N<br>• 对噪声敏感 | • 高维表示<br>• 找到对应关系 |
| **Procrustes** | • 几何直观<br>• 满足距离度量 | • 需要相同维度<br>• 仅正交变换 | • 同架构模型<br>• 需要几何解释 |

#### 函数相似度 (Functional Similarity):

| 方法 Method | 适用场景 Use Case | 注意事项 Notes |
|------------|------------------|---------------|
| **Disagreement** | 快速比较 | 受准确率影响 |
| **JSD** | 概率分布比较 | 考虑预测置信度 |
| **Gradient-based** | 深入分析决策边界 | 需要白盒访问 |

### 7.2 实践建议 (Practical Recommendations)

**✅ 推荐做法 (Recommended Practices):**
1. 同时使用表示和函数相似度 (Use both representational and functional similarity)
2. 使用多个测量方法交叉验证 (Use multiple measures for cross-validation)
3. 考虑数据的统计特性 (Consider statistical properties of data)
4. 报告置信区间 (Report confidence intervals)

**❌ 避免做法 (Practices to Avoid):**
1. 仅用单一测量方法 (Relying on a single measure)
2. 忽略不变性假设 (Ignoring invariance assumptions)
3. 在小样本上下结论 (Drawing conclusions from small samples)

### 7.3 未来研究方向 (Future Research Directions)

论文指出的开放问题 (Open challenges identified in the paper):

1. **可解释性 (Interpretability)**
   - 如何解释相似度分数的生物学意义?
   - How to interpret similarity scores biologically?

2. **鲁棒性 (Robustness)**
   - 如何使测量对噪声和异常值更鲁棒?
   - How to make measures more robust to noise?

3. **计算效率 (Computational Efficiency)**
   - 如何在大规模数据上高效计算?
   - How to compute efficiently on large-scale data?

4. **理论保证 (Theoretical Guarantees)**
   - 什么时候高表示相似度意味着高函数相似度?
   - When does high representational similarity imply functional similarity?

### 7.4 你的研究中的应用 (Applications in Your Research)

**对你的毒理学和组蛋白研究的启示 (Implications for toxicology and histone research):**

1. **比较不同毒物的作用机制**
   - 使用表示相似度判断是否共享分子通路
   - Use representational similarity to determine shared pathways

2. **验证模型的生物学合理性**
   - 检查模型学到的特征是否与已知生物学一致
   - Check if learned features align with known biology

3. **迁移学习的可行性**
   - 高相似度表明可以迁移知识
   - High similarity suggests knowledge transfer is feasible

4. **发现新的生物学关联**
   - 意外的高相似度可能揭示未知联系
   - Unexpected high similarity may reveal unknown connections

---
## 📚 参考资源 (References)

### 主要论文 (Main Paper):
Klabunde, M., Schumacher, T., Strohmaier, M., & Lemmerich, F. (2025). 
Similarity of Neural Network Models: A Survey of Functional and Representational Measures. 
*ACM Computing Surveys*, 57(9), Article 242.

### 相关工具和库 (Related Tools & Libraries):
- **ReSi Benchmark**: https://github.com/mklabunde/resi
- **Similarity Repository**: https://github.com/nacloos/similarity-repository
- **CKA Implementation**: Various implementations in PyTorch/TensorFlow

### 扩展阅读 (Further Reading):
1. Kornblith et al. (2019) - CKA原始论文 (Original CKA paper)
2. Raghu et al. (2017) - SVCCA
3. Morcos et al. (2018) - PWCCA

---

## 💡 练习和思考题 (Exercises & Questions)

1. **为什么CKA对正交变换不变,而Euclidean距离不是?**
   - Why is CKA invariant to orthogonal transformations but Euclidean distance is not?

2. **在什么情况下,两个模型的表示相似但输出不同?**
   - When can two models have similar representations but different outputs?

3. **如何为你的RNA-seq数据选择合适的相似度测量?**
   - How would you choose appropriate similarity measures for RNA-seq data?

4. **尝试修改代码,添加你自己的数据!**
   - Try modifying the code to add your own data!

---

## 🎯 下一步 (Next Steps)

1. 阅读完整论文了解更多细节
2. 在你的实际数据上测试这些方法
3. 探索ReSi benchmark
4. 考虑如何将这些方法应用到你的毒理学研究中

---

**Created by:** YueYueYYY  
**Date:** December 2025  
**Based on:** Klabunde et al. (2025) ACM Computing Surveys

**祝学习愉快! Happy Learning! 🚀**